In [ ]:
!pip install matplotlib
!pip install tensorflow
!pip install numpy<2
!pip install --upgrade ipywidgets jupyter
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.6 MB/s eta 0:00:00
/bin/bash: line 1: 2: No such file or directory
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install pandas

In [ ]:
!pip install scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor
from IPython.display import clear_output
from itertools import combinations
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, LSTM
from keras.models import Sequential
from keras.optimizers import Adam  # Import the Adam optimizer
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.layers import Dropout, Input
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.models import Sequential
from google.colab import drive


# Set pandas display options to avoid truncation
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows (if using .head() this will show all rows returned by .head())
pd.set_option('display.max_colwidth', None) # Show full column width for each column

In [ ]:
drive.mount('/content/drive', force_remount=True)
train_df = pd.read_csv('drive/MyDrive/training.csv')
test_df = pd.read_csv('drive/MyDrive/test.csv')

Mounted at /content/drive


In [ ]:
train = train_df.copy()
test = test_df.copy()

In [ ]:
#train.head(300)

In [ ]:
def create_validation_set(train_df, validation_fraction=0.2):
    # Step 1: Extract unique client IDs from the training set
    unique_ids = train_df['client_new_id'].unique()

    # Step 2: Randomly select 20% of these unique IDs for validation
    num_validation_ids = int(len(unique_ids) * validation_fraction)
    validation_ids = np.random.choice(unique_ids, size=num_validation_ids, replace=False)

    # Step 3: Create the validation set by filtering the rows with the selected IDs
    validation_set = train_df[train_df['client_new_id'].isin(validation_ids)]

    # Step 4: Drop the selected IDs from the training set
    training_set = train_df[~train_df['client_new_id'].isin(validation_ids)]

    return training_set, validation_set

In [ ]:
def process_and_reorder_df(df):
    def process_client_data(client_df):
        max_period = client_df['period'].max()
        operation_type = 'SAME'  # Default operation type

        if max_period < 10:
            # Padding: Add rows with all ones to make the sequence start from 1 to 10
            missing_periods = 10 - max_period
            # Create padding rows with all ones except for the 'client_id' and 'period' columns
            padding_rows = pd.DataFrame(1, index=range(missing_periods), columns=[col for col in client_df.columns if col not in ['client_id', 'period']])
            padding_rows['period'] = range(1, missing_periods + 1)
            padding_rows['client_id'] = client_df['client_id'].iloc[0]  # Copy the client_id for all padding rows

            client_df['period'] = client_df['period'] + missing_periods
            client_df = pd.concat([padding_rows, client_df], ignore_index=True)

            # Update the operation type
            operation_type = f'PADDED_{missing_periods}'

        elif max_period > 10:
            # Truncate: Keep only the last 10 periods
            length_of_trunc = max_period - 10
            client_df = client_df.tail(10)
            # Correct the periods to be in sequence from 1 to 10
            client_df['period'] = range(1, 11)

            # Update the operation type
            operation_type = f'TRUNC_{length_of_trunc}'

        # Add the operation type column to the DataFrame
        client_df['operation_type'] = operation_type

        # Reorder columns to place 'operation_type' as the second column
        cols = ['client_id', 'operation_type'] + [col for col in client_df.columns if col not in ['client_id', 'operation_type']]
        client_df = client_df[cols]

        return client_df

    # Apply the function to each client group
    df = df.groupby('client_id', group_keys=False).apply(process_client_data)

    return df

In [ ]:
# Apply StandardScaler to the selected columns
scaler = MinMaxScaler()

def scale(df, is_train=False):
    # List of columns to exclude from scaling
    exclude_columns = [
        'product8', 'product10', 'product13', 'product12', 'product11',
        'product4', 'product17', 'product2', 'product3', 'product1',
        'product7', 'product6', 'product5', 'product14', 'product15',
        'product16', 'product9', 'has_products',
        'left_bank', 'joined_bank', 'securities_bought', 'securities_sold',
        'repays_debt', 'period', 'client_id', 'client_new_id', 'operation_type', 'is_active'
    ]

    # Identify columns that start with 'interaction_'
    interaction_columns = [col for col in df.columns if col.startswith('interaction_')]
    exclude_columns.extend(interaction_columns)

    # Identify columns that start with 'category_'
    category_columns = [col for col in df.columns if col.startswith('category_')]
    exclude_columns.extend(category_columns)

    # Ensure the columns to exclude are unique
    exclude_columns = list(set(exclude_columns))

    # Determine columns to standardize
    columns_to_standardize = [col for col in df.columns if col not in exclude_columns]

    if is_train:
        scaler.fit(df[columns_to_standardize])

    # Option 1: Using .loc for assignment
    df.loc[:, columns_to_standardize] = scaler.transform(df[columns_to_standardize])

    return df

In [ ]:
def pipeline(df, is_test=False, is_train=False):
# List of product columns
    product_columns = [
        'product8', 'product10', 'product13', 'product12', 'product11',
        'product4', 'product17', 'product2', 'product3', 'product1',
        'product7', 'product6', 'product5', 'product14', 'product15',
        'product16', 'product9'
    ]

    epsilon = 1e-10  # Small value to prevent division by zero

    # Create a dictionary to store new columns
    new_columns = {}

    # Calculate total_products as the sum of all product columns
    new_columns['total_products'] = df[product_columns].sum(axis=1)

    # Calculate number of non-zero products
    new_columns['num_products'] = (df[product_columns] > 0).sum(axis=1)

    # Calculate inbound to outbound wire transfer ratio
    new_columns['inbound_to_outbound_wire_transfer_ratio'] = (
        (df['wire_transfers2_amt_inbound'] + df['wire_transfers1_amt_inbound']) /
        ((df['wire_transfers2_amt_outbound'] + df['wire_transfers1_amt_outbound']) + epsilon)
    )

    # Calculate total wire transfers amount
    new_columns['total_wire_transfers_amount'] = (
        df['wire_transfers2_amt_inbound'] + df['wire_transfers1_amt_inbound'] +
        df['wire_transfers2_amt_outbound'] + df['wire_transfers1_amt_outbound']
    )

    # Calculate total wire transfers count
    new_columns['total_wire_transfers_count'] = (
        df['wire_transfers2_num_inbound'] + df['wire_transfers1_num_inbound'] +
        df['wire_transfers2_num_outbound'] + df['wire_transfers1_num_outbound']
    )

    # Calculate net counter transactions
    new_columns['net_counter_transactions'] = df['counter_amt_inbound'] - df['counter_amt_outbound']

    # Calculate securities activity
    new_columns['securities_activity'] = df['securities_bought_amt'] + df['securities_sold_amt']

    # Calculate balance per product
    new_columns['balance_per_product'] = df['balance'] / (new_columns['total_products'] + epsilon)

    # Calculate total transactions
    new_columns['total_transactions'] = (
        df['counter_amt_inbound'] + df['counter_amt_outbound'] +
        df['wire_transfers2_amt_inbound'] + df['wire_transfers1_amt_inbound'] +
        df['wire_transfers2_amt_outbound'] + df['wire_transfers1_amt_outbound']
    )

    # Generate interaction terms
    #for (prod1, prod2) in combinations(product_columns, 2):
    #    interaction_col_name = f'interaction_{prod1}_{prod2}'
    #    new_columns[interaction_col_name] = df[prod1] * df[prod2]

    # Categorical Encoding
    category_dummies = pd.get_dummies(df['category'], prefix='category')
    df = df.drop(columns=['category']).join(category_dummies)

    # Add new columns to the DataFrame
    df = df.assign(**new_columns)

    # Rolling Aggregations
    df['rolling_balance_mean'] = df.groupby('client_id')['balance'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)

    # Client Activity Metrics
    df['is_active'] = df[product_columns + ['total_wire_transfers_amount', 'total_transactions']].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

    # Transaction Frequency
    df['avg_wire_transfers_per_period'] = df['total_wire_transfers_count'] / df['period']

    # Balance Stability
    df['balance_change_std'] = df.groupby('client_id')['balance'].transform(lambda x: x.diff().std()).fillna(0)

    # Transaction Types
    df['total_inbound_transactions'] = (
        df['wire_transfers2_amt_inbound'] + df['wire_transfers1_amt_inbound'] +
        df['counter_amt_inbound']
    )

    # Percentage of Transactions
    df['pct_inbound_wire_transfers'] = (
        (df['wire_transfers2_amt_inbound'] + df['wire_transfers1_amt_inbound']) /
        (df['total_wire_transfers_amount'] + epsilon)
    )

    # Product Utilization Ratio
    total_available_products = len(product_columns)
    df['product_utilization_ratio'] = new_columns['total_products'] / total_available_products

    # Transaction Amount per Product
    df['transaction_amount_per_product'] = df['total_transactions'] / (new_columns['total_products'] + epsilon)

    # Balance and Product Interaction
    df['balance_product_interaction'] = df['balance'] * new_columns['total_products']

    category_columns = ['category_1', 'category_2', 'category_3']
    df[category_columns] = df[category_columns].astype(int)

    if is_test:
        df = process_and_reorder_df(df)
    else:
        # Determining the current status of the account
        #df = determine_account_status(df)

        # Create the client_new_id column
        df['client_new_id'] = df['client_id'].astype(str) + df['period'].apply(lambda x: '_p1' if x <= 10 else '_p2')

        # Drop the original client_id column
        #df = df.drop(columns=['client_id'])

        # Reorder the columns to place 'client_new_id' and 'period' at the beginning
        cols = ['client_id', 'client_new_id', 'period'] + [col for col in df.columns if col not in ['client_id', 'client_new_id', 'period']]
        df = df[cols]

        # Adjust the period values
        df['period'] = df['period'].apply(lambda x: x if x <= 10 else x - 10)

    # Step 1: Identify all columns
    columns = list(df.columns)

    # Step 2: Remove 'repays_debt' from its current position
    columns.remove('repays_debt')

    # Step 3: Append 'repays_debt' to the end of the columns list
    columns.append('repays_debt')

    # Step 4: Reorder the DataFrame using the new columns list
    df = df[columns]

    return df

In [ ]:
transformed_train = pipeline(train, is_train=True)

clear_output(wait=True)

In [ ]:
vif_use = transformed_train.copy()
vif_use.drop(columns=['client_new_id', 'client_id', 'period', 'repays_debt'], inplace=True)
# Add a constant to the model (intercept)
X = sm.add_constant(vif_use)

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_data.head(80)

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1783: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


In [ ]:
transformed_train.head(100)

,client_id,client_new_id,period,product8,product10,product13,product12,product11,product4,product17,product2,product3,product1,product7,product6,product5,product14,product15,product16,product9,has_products,balance,left_bank,joined_bank,wire_transfers2_amt_inbound,wire_transfers1_amt_inbound,wire_transfers2_amt_outbound,wire_transfers1_amt_outbound,counter_amt_inbound,counter_amt_outbound,securities_bought_amt,securities_sold_amt,wire_transfers2_num_inbound,wire_transfers1_num_inbound,wire_transfers2_num_outbound,wire_transfers1_num_outbound,counter_num_inbound,counter_num_outbound,securities_operations,securities_bought,securities_sold,counter_amt_tot,counter_num_tot,category_1,category_2,category_3,total_products,num_products,inbound_to_outbound_wire_transfer_ratio,total_wire_transfers_amount,total_wire_transfers_count,net_counter_transactions,securities_activity,balance_per_product,total_transactions,rolling_balance_mean,is_active,avg_wire_transfers_per_period,balance_change_std,total_inbound_transactions,pct_inbound_wire_transfers,product_utilization_ratio,transaction_amount_per_product,balance_product_interaction,repays_debt
0,0,0_p1,1,1,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,1,1,-163332.16,0,0,0.000000,112212.879740,0.000000,-21795.269753,510046.938969,-317897.973147,0.0,0.0,0,12,0,12,45,61,0,0,0,192148.965822,106,0,1,0,6,6,-5.148497e+00,90417.609987,24,827944.912116,0.0,-2.722203e+04,282566.575809,-163332.160000,1,24.000000,53753.254724,622259.818709,1.241051,0.352941,4.709443e+04,-979992.96,0
1,0,0_p1,2,1,1,1,1,0,0,0,1,1,0,0,0,1,0,1,1,1,1,-156594.80,0,0,0.000000,94200.287118,0.000000,-14723.443860,291837.212999,-348179.227230,0.0,0.0,0,8,0,14,31,50,0,0,0,-56342.014232,81,0,1,0,10,10,-6.397979e+00,79476.843258,22,640016.440229,0.0,-1.565948e+04,23134.829026,-159963.480000,1,11.000000,53753.254724,386037.500117,1.185255,0.588235,2.313483e+03,-1565948.00,0
2,0,0_p1,3,1,0,1,0,0,1,0,0,1,1,0,0,0,0,1,0,1,1,-204463.36,0,0,0.000000,89195.413877,0.000000,-13275.698340,224838.362793,-384085.846269,0.0,0.0,0,7,0,13,16,47,0,0,0,-159247.483476,63,0,1,0,7,7,-6.718698e+00,75919.715537,20,608924.209062,0.0,-2.920905e+04,-83327.767938,-174796.773333,0,6.666667,53753.254724,314033.776671,1.174865,0.411765,-1.190397e+04,-1431243.52,0
3,0,0_p1,4,1,1,1,0,1,1,1,0,1,0,0,0,0,0,1,1,1,1,-142250.65,0,0,0.000000,167307.194913,0.000000,-51755.736333,330268.898579,-318033.979659,0.0,0.0,0,14,0,16,24,53,0,0,0,12234.918920,77,0,1,0,10,10,-3.232631e+00,115551.458580,30,648302.878238,0.0,-1.422506e+04,127786.377500,-167769.603333,1,7.500000,53753.254724,497576.093492,1.447902,0.588235,1.277864e+04,-1422506.50,0
4,0,0_p1,5,1,0,1,1,1,0,0,0,1,0,0,1,0,0,1,0,1,1,-153658.46,0,0,2840.732885,92703.009488,0.000000,-7679.854189,168909.669220,-273366.453144,0.0,0.0,1,13,0,10,24,53,0,0,0,-104456.783924,77,0,1,0,8,8,-1.244083e+01,87863.888183,24,442276.122364,0.0,-1.920731e+04,-16592.895741,-166790.823333,1,4.800000,53753.254724,264453.411592,1.087406,0.470588,-2.074112e+03,-1229267.68,0
5,0,0_p1,6,1,1,1,1,0,1,1,0,1,0,0,0,0,0,1,0,1,1,-112160.82,0,0,0.000000,46642.365407,0.000000,-10512.709463,218507.744368,-278809.396292,0.0,0.0,0,9,0,18,20,32,0,0,0,-60301.651924,52,0,1,0,9,9,-4.436760e+00,36129.655944,27,497317.140661,0.0,-1.246231e+04,-24171.995980,-136023.310000,1,4.500000,53753.254724,265150.109775,1.290972,0.529412,-2.685777e+03,-1009447.38,0
6,0,0_p1,7,1,0,1,0,1,0,0,0,0,0,0,1,1,0,1,0,1,1,-104242.27,0,0,0.000000,126566.975696,0.000000,-27026.889140,250907.585945,-330648.897581,0.0,0.0,0,12,0,18,31,42,0,0,0,-79741.311637,73,0,1,0,7,7,-4.683002e+00,99540.086555,30,581556.483526,0.0,-1.489175e+04,19798.774919,-123353.850000,1,4.285714,53753.254724,377474.561640,1.271518,0.411765,2.828396e+03,-729695.89,0
7,0,0_p1,8,1,1,0,0,1,0,0,0,1,1,0,1,1,0,1,0,1,1,-47317.27,0,0,0.000000,179942.809884,0.000000,-29145.718149,233273.478819,-280359.653924,0.0,0.0,0,13,0,22,18,32,0,0,0,-47086.175105,50,0,1,0,9,9,-6.173902e+00,150797.091735,35,513633.132743,0.0,-5.257474e+03,103710.916630,-879

In [ ]:
vif_use = transformed_train.copy()
vif_use.drop(columns=['client_new_id'], inplace=True)
# Add a constant to the model (intercept)
X = sm.add_constant(vif_use)

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_data.head(80)

/opt/conda/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1783: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


In [ ]:
final_train_set, final_val_set = create_validation_set(transformed_train)
final_train_set = scale(final_train_set, is_train=True)
final_val_set = scale(final_val_set, is_train=False)


/tmp/ipykernel_10126/2685808602.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, columns_to_standardize] = scaler.transform(df[columns_to_standardize])
/tmp/ipykernel_10126/2685808602.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.00039671 0.00026447 0.00023141 ... 0.         0.         0.        ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, columns_to_standardize] = scaler.transform(df[columns_to_standardize])
/tmp/ipykernel_10126/2685808602.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has dtype incompatible with int64, please explicitly cast to a com

In [ ]:
final_train_set.head(100)

,client_id,client_new_id,period,product8,product10,product13,product12,product11,product4,product17,product2,product3,product1,product7,product6,product5,product14,product15,product16,product9,has_products,balance,left_bank,joined_bank,wire_transfers2_amt_inbound,wire_transfers1_amt_inbound,wire_transfers2_amt_outbound,wire_transfers1_amt_outbound,counter_amt_inbound,counter_amt_outbound,securities_bought_amt,securities_sold_amt,wire_transfers2_num_inbound,wire_transfers1_num_inbound,wire_transfers2_num_outbound,wire_transfers1_num_outbound,counter_num_inbound,counter_num_outbound,securities_operations,securities_bought,securities_sold,counter_amt_tot,counter_num_tot,category_1,category_2,category_3,total_products,num_products,inbound_to_outbound_wire_transfer_ratio,total_wire_transfers_amount,total_wire_transfers_count,net_counter_transactions,securities_activity,balance_per_product,total_transactions,rolling_balance_mean,is_active,avg_wire_transfers_per_period,balance_change_std,total_inbound_transactions,pct_inbound_wire_transfers,product_utilization_ratio,transaction_amount_per_product,balance_product_interaction,repays_debt
0,0,0_p1,1,1,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,1,1,0.273626,0,0,0.000000,3.533067e-03,1.0,0.998808,0.008248,0.996973,0.0,0.0,0.000000,0.000397,0.0,0.004149,0.027456,0.087268,0.0,0,0,0.630972,0.061880,0,1,0,0.400000,0.400000,5.451755e-11,0.388976,0.000754,7.882530e-03,0.0,0.046208,0.634155,0.332963,1,0.000839,0.011872,1.004338e-02,0.823993,0.400000,0.599870,0.237499,0
1,0,0_p1,2,1,1,1,1,0,0,0,1,1,0,0,0,1,0,1,1,1,1,0.274026,0,0,0.000000,2.965933e-03,1.0,0.999195,0.004719,0.996685,0.0,0.0,0.000000,0.000264,0.0,0.004841,0.018914,0.071531,0.0,0,0,0.629482,0.047285,0,1,0,0.666667,0.666667,5.451752e-11,0.388723,0.000691,6.093339e-03,0.0,0.046208,0.632612,0.333215,1,0.000385,0.011872,6.230708e-03,0.823993,0.666667,0.599870,0.232463,0
2,0,0_p1,3,1,0,1,0,0,1,0,0,1,1,0,0,0,0,1,0,1,1,0.271183,0,0,0.000000,2.808353e-03,1.0,0.999274,0.003636,0.996343,0.0,0.0,0.000000,0.000231,0.0,0.004495,0.009762,0.067239,0.0,0,0,0.628865,0.036778,0,1,0,0.466667,0.466667,5.451751e-11,0.388641,0.000628,5.797322e-03,0.0,0.046208,0.631978,0.332104,0,0.000233,0.011872,5.068556e-03,0.823992,0.466667,0.599870,0.233621,0
3,0,0_p1,4,1,1,1,0,1,1,1,0,1,0,0,0,0,0,1,1,1,1,0.274878,0,0,0.000000,5.267733e-03,1.0,0.997169,0.005341,0.996972,0.0,0.0,0.000000,0.000463,0.0,0.005533,0.014643,0.075823,0.0,0,0,0.629893,0.044950,0,1,0,0.666667,0.666667,5.451760e-11,0.389556,0.000942,6.172231e-03,0.0,0.046208,0.633234,0.332630,1,0.000262,0.011872,8.030959e-03,0.823995,0.666667,0.599870,0.233696,0
4,0,0_p1,5,1,0,1,1,1,0,0,0,1,0,0,1,0,0,1,0,1,1,0.274201,0,0,0.001910,2.918791e-03,1.0,0.999580,0.002731,0.997397,0.0,0.0,0.007407,0.000430,0.0,0.003458,0.014643,0.075823,0.0,0,0,0.629193,0.044950,0,1,0,0.533333,0.533333,5.451736e-11,0.388917,0.000754,4.210733e-03,0.0,0.046208,0.632375,0.332704,1,0.000168,0.011872,4.268321e-03,0.823992,0.533333,0.599870,0.235357,0
5,0,0_p1,6,1,1,1,1,0,1,1,0,1,0,0,0,0,0,1,0,1,1,0.276665,0,0,0.000000,1.468553e-03,1.0,0.999425,0.003533,0.997345,0.0,0.0,0.000000,0.000298,0.0,0.006224,0.012203,0.045780,0.0,0,0,0.629458,0.030356,0,1,0,0.600000,0.600000,5.451757e-11,0.387724,0.000848,4.734756e-03,0.0,0.046208,0.632330,0.335008,1,0.000157,0.011872,4.279566e-03,0.823994,0.600000,0.599870,0.237246,0
6,0,0_p1,7,1,0,1,0,1,0,0,0,0,0,0,1,1,0,1,0,1,1,0.277136,0,0,0.000000,3.985011e-03,1.0,0.998522,0.004057,0.996852,0.0,0.0,0.000000,0.000397,0.0,0.006224,0.018914,0.060086,0.0,0,0,0.629341,0.042615,0,1,0,0.466667,0.466667,5.451756e-11,0.389186,0.000942,5.536765e-03,0.0,0.046208,0.632592,0.335957,1,0.000150,0.011872,6.092501e-03,0.823993,0.466667,0.599870,0.239650,0
7,0,0_p1,8,1,1,0,0,1,0,0,0,1,1,0,1,1,0,1,0,1,1,0.280517,0,0,0.000000,5.665571e-03,1.0,0.998406,0.003772,0.997331,0.0,0.0,0.000000,0.000430,0.0,0.007607,0.010982,0.045780,0.0,0,0,0.629537,0.029189,0,1,0,0.600000,0.600000,5.451752e-11,0.390369,0.001099,4.890094e-03,0.0,0.046208,0.633091,0.338

In [ ]:
final_val_set.head(100)

,client_id,client_new_id,period,product8,product10,product13,product12,product11,product4,product17,product2,product3,product1,product7,product6,product5,product14,product15,product16,product9,has_products,balance,left_bank,joined_bank,wire_transfers2_amt_inbound,wire_transfers1_amt_inbound,wire_transfers2_amt_outbound,wire_transfers1_amt_outbound,counter_amt_inbound,counter_amt_outbound,securities_bought_amt,securities_sold_amt,wire_transfers2_num_inbound,wire_transfers1_num_inbound,wire_transfers2_num_outbound,wire_transfers1_num_outbound,counter_num_inbound,counter_num_outbound,securities_operations,securities_bought,securities_sold,counter_amt_tot,counter_num_tot,category_1,category_2,category_3,total_products,num_products,inbound_to_outbound_wire_transfer_ratio,total_wire_transfers_amount,total_wire_transfers_count,net_counter_transactions,securities_activity,balance_per_product,total_transactions,rolling_balance_mean,is_active,avg_wire_transfers_per_period,balance_change_std,total_inbound_transactions,pct_inbound_wire_transfers,product_utilization_ratio,transaction_amount_per_product,balance_product_interaction,repays_debt
40,2,2_p1,1,1,1,0,0,1,0,1,1,1,1,0,0,0,0,0,1,0,1,0.285343,0,0,0.000000,4.282956e-05,0.939404,0.999294,0.002433,0.998796,0.0,0.0,0.000000,0.000099,0.017857,0.005878,0.040879,0.038627,0.0,0,0,0.629964,0.054874,0,1,0,0.533333,0.533333,5.451768e-11,0.385710,0.000660,0.002636,0.0,0.046208,0.632312,0.347738,0,0.000734,0.004833,0.002451,0.823980,0.533333,0.59987,0.248254,0
41,2,2_p1,2,0,1,1,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,0.283069,0,0,0.000000,7.074060e-05,0.946281,0.998723,0.001910,0.998950,0.0,0.0,0.000000,0.000066,0.017857,0.007261,0.042099,0.045780,0.0,0,0,0.629867,0.058961,0,1,0,0.466667,0.466667,5.451768e-11,0.385593,0.000754,0.002175,0.0,0.046208,0.632186,0.346304,0,0.000419,0.004833,0.001943,0.823980,0.466667,0.59987,0.245659,0
42,2,2_p1,3,1,1,1,0,0,1,0,1,1,1,0,0,0,0,0,0,0,1,0.284348,0,0,0.000000,1.234786e-03,1.000000,0.999008,0.002309,0.999485,0.0,0.0,0.000000,0.000430,0.000000,0.005533,0.039048,0.044349,0.0,0,0,0.630352,0.055458,0,1,0,0.466667,0.466667,5.451763e-11,0.387376,0.000911,0.001874,0.0,0.046208,0.633127,0.346364,1,0.000338,0.004833,0.002938,0.824000,0.466667,0.59987,0.246954,0
43,2,2_p1,4,1,0,1,0,0,0,1,1,1,1,1,1,0,1,1,0,1,1,0.283240,0,0,0.000000,1.120784e-04,0.960582,0.998883,0.001755,0.999346,0.0,0.0,0.000000,0.000198,0.017857,0.006224,0.032337,0.038627,0.0,0,0,0.630059,0.046702,0,1,0,0.733333,0.733333,5.451768e-11,0.385907,0.000785,0.001687,0.0,0.046208,0.632458,0.345480,0,0.000218,0.004833,0.001809,0.823979,0.733333,0.59987,0.245781,0
44,2,2_p1,5,1,0,1,0,0,0,0,1,1,1,0,0,0,1,1,0,0,1,0.282958,0,0,0.000000,9.021074e-04,0.953754,0.999189,0.002102,0.998969,0.0,0.0,0.000000,0.000364,0.017857,0.005533,0.037828,0.042918,0.0,0,0,0.629950,0.053707,0,1,0,0.466667,0.466667,5.451767e-11,0.386511,0.000880,0.002268,0.0,0.046208,0.632505,0.345433,0,0.000196,0.004833,0.002560,0.823962,0.466667,0.59987,0.245546,0
45,2,2_p1,6,1,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0.284599,0,0,0.000000,1.257215e-04,1.000000,0.998458,0.001850,0.999314,0.0,0.0,0.000000,0.000331,0.000000,0.006570,0.024405,0.037196,0.0,0,0,0.630074,0.038529,0,1,0,0.400000,0.400000,5.451768e-11,0.386332,0.000911,0.001775,0.0,0.046208,0.632582,0.345538,0,0.000169,0.004833,0.001911,0.823978,0.400000,0.59987,0.247024,0
46,2,2_p1,7,0,1,1,0,0,0,0,0,1,1,0,0,0,0,1,1,0,1,0.283623,0,0,0.000000,1.917061e-04,1.000000,0.999232,0.001708,0.999392,0.0,0.0,0.000000,0.000331,0.000000,0.006224,0.034777,0.045780,0.0,0,0,0.630070,0.051956,0,1,0,0.400000,0.400000,5.451767e-11,0.386707,0.000880,0.001613,0.0,0.046208,0.632675,0.345700,1,0.000140,0.004833,0.001803,0.823972,0.400000,0.59987,0.246177,0
47,2,2_p1,8,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0.282870,0,0,0.000000,1.535742e-04,1.000000,0.999220,0.001676,0.999603,0.0,0.0,0.000000,0.000364,0.000000,0.005533,0.024405,0.028612,0.0,0,0,0.630191,0.035026,0,1,0,0.266667,0.266667,5.451768e-11,0.386674,0.000848,0.001383,0.0,0.

In [ ]:
transformed_test = pipeline(test, is_test=True)
transformed_test = scale(transformed_test, is_train=False)
clear_output(wait=True)

/tmp/ipykernel_10126/3356180132.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('client_id', group_keys=False).apply(process_client_data)


In [ ]:
transformed_test.head(100)

,client_id,operation_type,product8,product10,product13,product12,product11,product4,product17,product2,product3,product1,product7,product6,product5,product14,product15,product16,product9,has_products,balance,left_bank,joined_bank,wire_transfers2_amt_inbound,wire_transfers1_amt_inbound,wire_transfers2_amt_outbound,wire_transfers1_amt_outbound,counter_amt_inbound,counter_amt_outbound,securities_bought_amt,securities_sold_amt,wire_transfers2_num_inbound,wire_transfers1_num_inbound,wire_transfers2_num_outbound,wire_transfers1_num_outbound,counter_num_inbound,counter_num_outbound,securities_operations,securities_bought,securities_sold,counter_amt_tot,counter_num_tot,period,category_1,category_2,category_3,total_products,num_products,inbound_to_outbound_wire_transfer_ratio,total_wire_transfers_amount,total_wire_transfers_count,net_counter_transactions,securities_activity,balance_per_product,total_transactions,rolling_balance_mean,is_active,avg_wire_transfers_per_period,balance_change_std,total_inbound_transactions,pct_inbound_wire_transfers,product_utilization_ratio,transaction_amount_per_product,balance_product_interaction,repays_debt
0,1,SAME,0,0,0,0,1,0,1,1,0,0,1,0,1,1,0,0,0,0,0.283327,0,0,0.000000e+00,0.000000e+00,1.000000,1.000000,0.000000e+00,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.307967,0.000000,1,0,0,1,0.400000,0.400000,5.664962e-11,0.655691,0.000000,0.000000e+00,0.000000e+00,0.030878,0.347219,0.332713,1,0.000000,0.000000e+00,0.000000e+00,0.877015,0.400000,0.947534,0.245920,0
1,1,SAME,1,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0.283327,0,0,0.000000e+00,0.000000e+00,1.000000,1.000000,0.000000e+00,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.307967,0.000000,2,0,0,1,0.333333,0.333333,5.664962e-11,0.655691,0.000000,0.000000e+00,0.000000e+00,0.030878,0.347219,0.332713,1,0.000000,0.000000e+00,0.000000e+00,0.877015,0.333333,0.947534,0.245920,0
2,1,SAME,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0.283327,0,0,0.000000e+00,0.000000e+00,1.000000,1.000000,0.000000e+00,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.307967,0.000000,3,0,0,1,0.200000,0.200000,5.664962e-11,0.655691,0.000000,0.000000e+00,0.000000e+00,0.030878,0.347219,0.332713,1,0.000000,0.000000e+00,0.000000e+00,0.877015,0.200000,0.947534,0.245920,0
3,1,SAME,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.283327,0,0,0.000000e+00,0.000000e+00,1.000000,1.000000,0.000000e+00,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.307967,0.000000,4,0,0,1,0.200000,0.200000,5.664962e-11,0.655691,0.000000,0.000000e+00,0.000000e+00,0.030878,0.347219,0.332713,1,0.000000,0.000000e+00,0.000000e+00,0.877015,0.200000,0.947534,0.245920,0
4,1,SAME,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0.283327,0,0,0.000000e+00,0.000000e+00,1.000000,1.000000,0.000000e+00,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.307967,0.000000,5,0,0,1,0.133333,0.133333,5.664962e-11,0.655691,0.000000,0.000000e+00,0.000000e+00,0.030878,0.347219,0.332713,1,0.000000,0.000000e+00,0.000000e+00,0.877015,0.133333,0.947534,0.245920,0
5,1,SAME,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0.283327,0,0,0.000000e+00,0.000000e+00,1.000000,1.000000,0.000000e+00,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.307967,0.000000,6,0,0,1,0.133333,0.133333,5.664962e-11,0.655691,0.000000,0.000000e+00,0.000000e+00,0.030878,0.347219,0.332713,1,0.000000,0.000000e+00,0.000000e+00,0.877015,0.133333,0.947534,0.245920,0
6,1,SAME,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.283327,0,0,0.000000e+00,0.000000e+00,1.000000,1.000000,0.000000e+00,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.307967,0.000000,7,0,0,1,0.066667,0.066667,5.664962e-11,0.655691,0.000000,0.000000e+00,0.000000e+00,0.030878,0.347

In [ ]:
print(transformed_train.shape)
print(transformed_test.shape)

(2056160, 65)
(7180, 65)


In [ ]:
print(final_train_set.shape)
print(final_val_set.shape)

(1644930, 65)
(411230, 65)


In [ ]:
# Step 1: Create "train_targets" and "val_targets" using rows with period = 10
# Step 1: Filter rows where period = 10
train_period_10 = final_train_set[final_train_set['period'] == 10]
val_period_10 = final_val_set[final_val_set['period'] == 10]

# Step 3: Convert to NumPy array
feature_columns = ['repays_debt']

# Convert the filtered DataFrame to NumPy array
train_targets = train_period_10[feature_columns].values
val_targets = val_period_10[feature_columns].values

# Reshape to (num_clients, 1, num_features)
#train_targets = train_targets.reshape((train_targets.shape[0], 1, train_targets.shape[1]))
#val_targets = val_targets.reshape((val_targets.shape[0], 1, val_targets.shape[1]))

# Step 2: Eliminate rows with period = 10
filtered_train_set = final_train_set[final_train_set['period'] != 10]
filtered_validation_set = final_val_set[final_val_set['period'] != 10]

# Step 3: Drop columns
columns_to_drop = ['client_id', 'client_new_id', 'period', 'repays_debt']
filtered_train_set = filtered_train_set.drop(columns=columns_to_drop)
filtered_validation_set = filtered_validation_set.drop(columns=columns_to_drop)

filtered_train_np = filtered_train_set.to_numpy()
filtered_val_np = filtered_validation_set.to_numpy()

print("---Starting train file save---")
train_3d_array = filtered_train_np.reshape(-1, 9, filtered_train_np.shape[-1])
print("---Starting val file save---")
val_3d_array = filtered_val_np.reshape(-1, 9, filtered_val_np.shape[-1])

---Starting train file save---
---Starting val file save---


In [ ]:
print(train_3d_array.shape)
print(val_3d_array.shape)
print(train_targets.shape)
print(val_targets.shape)

(164493, 9, 61)
(41123, 9, 61)
(164493, 1)
(41123, 1)


In [ ]:
target_train_last = np.squeeze(train_targets[:, -1])
target_val_last = np.squeeze(val_targets[:, -1])

In [ ]:
print(target_train_last.shape)
print(target_val_last.shape)

(164493,)
(41123,)


In [ ]:
# Assuming target_val_last is a NumPy array
unique_values, counts = np.unique(target_val_last, return_counts=True)

# Display the unique values and their counts
for value, count in zip(unique_values, counts):
    print(f"Value: {value}, Count: {count}")

Value: 0, Count: 41003
Value: 1, Count: 120


In [ ]:
# Assuming target_val_last is a NumPy array
unique_values, counts = np.unique(target_train_last, return_counts=True)

# Display the unique values and their counts
for value, count in zip(unique_values, counts):
    print(f"Value: {value}, Count: {count}")

Value: 0, Count: 164055
Value: 1, Count: 438


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Masking, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import AUC, Precision, Recall, TruePositives, TrueNegatives, FalsePositives, FalseNegatives

def build_model(input_shape, dropout_rate=0.3):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Masking(mask_value=1))

    # First LSTM layer
    model.add(LSTM(units=128, return_sequences=True, recurrent_dropout=dropout_rate))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    # Second LSTM layer
    model.add(LSTM(units=64, return_sequences=True, recurrent_dropout=dropout_rate))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    # Third LSTM layer
    model.add(LSTM(units=32, return_sequences=False, recurrent_dropout=dropout_rate))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    # Fully connected layers
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(dropout_rate))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            AUC(name='auc'),
            Precision(name='precision'),
            Recall(name='recall'),
            TruePositives(name='tp'),
            TrueNegatives(name='tn'),
            FalsePositives(name='fp'),
            FalseNegatives(name='fn')
        ]
    )

    return model


# Callbacks
def get_callbacks():
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
    return [reduce_lr, early_stopping, checkpoint]

# Assuming x_train, y_train, x_val, y_val are already defined
input_shape = (train_3d_array.shape[1], train_3d_array.shape[2])
model = build_model(input_shape)
callbacks = get_callbacks()

# Assuming `train_targets` is the array with the binary labels (0 and 1)
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(target_train_last),
                                     y=target_train_last.flatten())

# Convert to a dictionary format that Keras expects
class_weight_dict = dict(enumerate(class_weights))
print(f"Calculated class weights: {class_weight_dict}")

# Train the Model
history = model.fit(train_3d_array, target_train_last, epochs=50, batch_size=64,
                    class_weight=class_weight_dict,
                    validation_data=(val_3d_array, target_val_last),
                    callbacks=callbacks)

# Load the best weights from the checkpoint
model.load_weights('best_model.keras')

# Make Predictions
preds = model.predict(val_3d_array)

# Convert predictions to binary based on a threshold
def threshold_predictions(predictions, threshold=0.5):
    return (predictions > threshold).astype(int)

# Extract true labels from y_val
y_true = np.squeeze(target_val_last)  # Adjust if y_val is (num_samples, 1)

# Calculate metrics
def compute_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    pr_auc = average_precision_score(y_true, y_pred)
    return precision, recall, f1, roc_auc, pr_auc

# Find the best threshold
def find_best_threshold(predictions, y_true):
    thresholds = np.arange(0.0, 1.1, 0.01)
    best_threshold = 0.0
    best_f1_score = 0.0

    for threshold in thresholds:
        predicted_classes = threshold_predictions(predictions, threshold)
        f1 = f1_score(y_true, predicted_classes)

        if f1 > best_f1_score:
            best_f1_score = f1
            best_threshold = threshold

    return best_threshold, best_f1_score

In [ ]:
print(np.squeeze(preds).shape)
print(y_true.shape)
print(np.squeeze(preds))
print(y_true)
# Get best threshold and F1 score
best_threshold, best_f1 = find_best_threshold(np.squeeze(preds), y_true)

# Apply best threshold
predicted_classes = threshold_predictions(preds, best_threshold)

# Compute and print metrics
precision, recall, f1, roc_auc, pr_auc = compute_metrics(y_true, predicted_classes)
print(f"Best Threshold: {best_threshold:.2f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"PR-AUC: {pr_auc:.4f}")

In [ ]:
# Step 2: Eliminate rows with period = 10
filtered_test_set = transformed_test[transformed_test['period'] != 10]

# Step 3: Drop columns
columns_to_drop = ['client_id', 'period', 'repays_debt', 'operation_type']
final_test_set = filtered_test_set.drop(columns=columns_to_drop)

final_test_set = final_test_set.to_numpy()

print("---Starting test file save---")
test_3d_array = final_test_set.reshape(-1, 9, filtered_train_np.shape[-1])

# Make Predictions
test_preds = model.predict(test_3d_array)

# Apply best threshold
predicted_classes = threshold_predictions(preds, 0.5)

In [ ]:
# Function to convert array to CSV
def array_to_csv(arr, filename="sub.csv"):
    with open(filename, 'w') as f:
        f.write("Label")  # Write header
        for value in arr.flatten():
            f.write(f"\n{value}")  # Write each value on a new line

    # Verify file length
    with open(filename, 'r') as f:
        line_count = sum(1 for line in f)
    print(f"File created with {line_count} lines (including header)")

# Convert array to CSV
array_to_csv(predicted_classes, "sub.csv")

